# Soft Impute 

In [1]:
import numpy as np
from fancyimpute import SoftImpute
import pandas as pd

### Create throughput df

In [2]:
# Carregar o dataset (ajustar o caminho para o seu arquivo)
df = pd.read_csv("institution_subnets/institution_subnets/agg_1_hour/0.csv")

# Exemplo de como o seu dataframe pode ser estruturado:
# df.columns = ['id_time', 'n_flows', 'n_packets', 'n_bytes', 'n_dest_ip', 'n_dest_asn', 'n_dest_port',
#               'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 
#               'avg_duration', 'avg_ttl']

# Calcular o throughput para uma janela de tempo de 1 hora
# Vamos supor que 'id_time' seja o identificador de cada intervalo de tempo e que os dados são agregados por hora

# Agrupar os dados por intervalo de uma hora
# Caso 'id_time' não seja um timestamp, você precisará convertê-lo em um formato temporal adequado

# Exemplo de uma simples conversão (se necessário):
# df['id_time'] = pd.to_datetime(df['id_time'], format='%Y-%m-%d %H:%M:%S')

# Calcular o throughput por hora (em bytes por segundo ou bps)
# Vamos assumir que 'n_bytes' seja o total de bytes transmitidos e que cada intervalo é de 1 hora (3600 segundos)
df['throughput'] = df['n_bytes'] / 3600 * 8  # Para throughput em bps

# Se precisar do throughput em Bps (bytes por segundo), basta remover o "* 8":
# df['throughput_bps'] = df['n_bytes'] / 3600  # Para throughput em bytes por segundo

# Criar um novo DataFrame apenas com a coluna 'throughput_bps'
df_throughput = df[['throughput']]

In [3]:
df_throughput.head()

,throughput
0,3.293856e+07
1,4.554900e+07
2,6.065814e+07
3,6.822189e+07
4,6.782679e+07


### Create "Temporal Matrix"

In [4]:
throughput_vector = df['throughput'].values

data_len = len(throughput_vector)
print(f"Throughput data length = {data_len}")

columns_quantity = data_len // 60 # one week 

Throughput data length = 6717


In [5]:
temporal_matrix = throughput_vector[:columns_quantity*60].reshape(columns_quantity, 60).T
original_df = pd.DataFrame(temporal_matrix)
print(original_df.shape)

(60, 111)


In [6]:
original_df.head()

,0,1,2,3,4,5,6,7,8,9,...,101,102,103,104,105,106,107,108,109,110
0,3.293856e+07,1.758036e+08,4.394803e+07,1.413289e+08,1.942840e+07,8.427017e+07,3.218003e+07,1.461119e+08,6.106981e+07,1.620767e+08,...,1.385539e+08,3.110653e+07,1.101810e+08,3.496407e+07,1.085399e+08,2.827794e+07,9.088500e+07,1.723778e+07,5.661395e+07,2.640076e+07
1,4.554900e+07,1.844747e+08,3.613200e+07,1.638002e+08,1.715774e+07,8.124855e+07,4.130530e+07,1.652897e+08,4.720167e+07,1.600402e+08,...,1.598589e+08,4.129183e+07,1.245614e+08,3.700972e+07,1.104097e+08,2.132867e+07,9.683220e+07,1.824239e+07,6.194095e+07,2.560527e+07
2,6.065814e+07,2.291110e+08,2.314620e+07,2.318234e+08,3.046354e+07,9.162349e+07,4.651750e+07,2.105914e+08,4.989807e+07,2.088379e+08,...,1.663969e+08,4.877947e+07,1.322101e+08,5.018397e+07,1.157455e+08,5.019845e+07,9.673623e+07,2.201507e+07,6.926160e+07,2.987725e+07
3,6.822189e+07,2.758808e+08,3.749324e+07,2.654625e+08,3.748267e+07,1.111421e+08,5.518027e+07,2.699339e+08,3.662139e+07,2.637365e+08,...,1.709646e+08,5.984108e+07,1.264215e+08,8.056551e+07,1.354795e+08,4.856147e+07,9.492283e+07,3.091887e+07,7.457657e+07,3.857762e+07
4,6.782679e+07,2.654722e+08,5.826158e+07,2.990796e+08,6.113911e+07,1.549960e+08,7.623457e+07,2.873457e+08,5.729947e+07,2.525232e+08,...,2.042090e+08,7.273202e+07,1.419164e+08,8.608843e+07,1.271596e+08,5.649672e+07,9.668526e+07,4.663450e+07,7.871942e+07,4.549146e+07


In [7]:
imputer = SoftImpute(max_iters=1000, shrinkage_value=0.1)

In [8]:
random_state = 42
df_missing = original_df.copy()
missing_quantity = int(0.2*data_len) # quantity of points missing 
nan_indexes = np.random.default_rng(random_state).choice(
    df_missing.size, missing_quantity, replace=False
)

nan_rows = nan_indexes // df_missing.shape[1] 
nan_cols = nan_indexes % df_missing.shape[1]

df_missing.values[nan_rows, nan_cols] = np.nan

In [9]:
nan_mask = df_missing.isna()
nan_mask.head()

,0,1,2,3,4,5,6,7,8,9,...,101,102,103,104,105,106,107,108,109,110
0,False,False,False,False,False,False,False,False,False,False,...,False,True,True,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,True,False,False,False,False,True,...,True,True,False,False,False,True,False,False,False,False
3,False,False,False,False,False,True,False,False,True,False,...,False,False,True,False,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [10]:
df_missing.head()

,0,1,2,3,4,5,6,7,8,9,...,101,102,103,104,105,106,107,108,109,110
0,3.293856e+07,1.758036e+08,4.394803e+07,1.413289e+08,1.942840e+07,8.427017e+07,3.218003e+07,1.461119e+08,6.106981e+07,1.620767e+08,...,1.385539e+08,NaN,NaN,3.496407e+07,1.085399e+08,2.827794e+07,9.088500e+07,NaN,5.661395e+07,2.640076e+07
1,4.554900e+07,1.844747e+08,3.613200e+07,1.638002e+08,1.715774e+07,8.124855e+07,4.130530e+07,1.652897e+08,4.720167e+07,1.600402e+08,...,1.598589e+08,4.129183e+07,1.245614e+08,3.700972e+07,1.104097e+08,2.132867e+07,9.683220e+07,1.824239e+07,6.194095e+07,2.560527e+07
2,NaN,2.291110e+08,2.314620e+07,2.318234e+08,NaN,9.162349e+07,4.651750e+07,2.105914e+08,4.989807e+07,NaN,...,NaN,NaN,1.322101e+08,5.018397e+07,1.157455e+08,NaN,9.673623e+07,2.201507e+07,6.926160e+07,2.987725e+07
3,6.822189e+07,2.758808e+08,3.749324e+07,2.654625e+08,3.748267e+07,NaN,5.518027e+07,2.699339e+08,NaN,2.637365e+08,...,1.709646e+08,5.984108e+07,NaN,8.056551e+07,NaN,4.856147e+07,9.492283e+07,3.091887e+07,7.457657e+07,3.857762e+07
4,6.782679e+07,2.654722e+08,5.826158e+07,2.990796e+08,6.113911e+07,1.549960e+08,7.623457e+07,2.873457e+08,5.729947e+07,2.525232e+08,...,2.042090e+08,7.273202e+07,1.419164e+08,8.608843e+07,1.271596e+08,NaN,9.668526e+07,4.663450e+07,7.871942e+07,4.549146e+07


In [11]:
X_filled_softimpute = imputer.fit_transform(df_missing)

c:\Users\macla\Documents\Projetos Dev\svd-cesnet\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\macla\Documents\Projetos Dev\svd-cesnet\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\macla\Documents\Projetos Dev\svd-cesnet\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[SoftImpute] Max Singular Value of X_init = 7524992485.601967
[SoftImpute] Iter 1: observed MAE=0.005934 rank=60
[SoftImpute] Iter 2: observed MAE=0.005934 rank=60
[SoftImpute] Iter 3: observed MAE=0.005934 rank=60
[SoftImpute] Iter 4: observed MAE=0.005934 rank=60
[SoftImpute] Iter 5: observed MAE=0.005934 rank=60
[SoftImpute] Iter 6: observed MAE=0.005934 rank=60
[SoftImpute] Iter 7: observed MAE=0.005934 rank=60
[SoftImpute] Iter 8: observed MAE=0.005934 rank=60
[SoftImpute] Iter 9: observed MAE=0.005934 rank=60
[SoftImpute] Iter 10: observed MAE=0.005934 rank=60
[SoftImpute] Iter 11: observed MAE=0.005934 rank=60
[SoftImpute] Iter 12: observed MAE=0.005934 rank=60
[SoftImpute] Iter 13: observed MAE=0.005934 rank=60
[SoftImpute] Iter 14: observed MAE=0.005934 rank=60
[SoftImpute] Iter 15: observed MAE=0.005934 rank=60
[SoftImpute] Iter 16: observed MAE=0.005934 rank=60
[SoftImpute] Iter 17: observed MAE=0.005934 rank=60
[SoftImpute] Iter 18: observed MAE=0.005934 rank=60
[SoftImpute

In [12]:
imputed_df = pd.DataFrame(X_filled_softimpute)
imputed_df.head()

,0,1,2,3,4,5,6,7,8,9,...,101,102,103,104,105,106,107,108,109,110
0,3.293856e+07,1.758036e+08,4.394803e+07,1.413289e+08,1.942840e+07,8.427017e+07,3.218003e+07,1.461119e+08,6.106981e+07,1.620767e+08,...,1.385539e+08,7.197137e+00,1.551993e+01,3.496407e+07,1.085399e+08,2.827794e+07,9.088500e+07,6.478788e+00,5.661395e+07,2.640076e+07
1,4.554900e+07,1.844747e+08,3.613200e+07,1.638002e+08,1.715774e+07,8.124855e+07,4.130530e+07,1.652897e+08,4.720167e+07,1.600402e+08,...,1.598589e+08,4.129183e+07,1.245614e+08,3.700972e+07,1.104097e+08,2.132867e+07,9.683220e+07,1.824239e+07,6.194095e+07,2.560527e+07
2,1.464097e+00,2.291110e+08,2.314620e+07,2.318234e+08,6.105126e+00,9.162349e+07,4.651750e+07,2.105914e+08,4.989807e+07,3.007496e+01,...,2.305604e+01,3.454422e-01,1.322101e+08,5.018397e+07,1.157455e+08,2.277677e+00,9.673623e+07,2.201507e+07,6.926160e+07,2.987725e+07
3,6.822189e+07,2.758808e+08,3.749324e+07,2.654625e+08,3.748267e+07,2.395277e+01,5.518027e+07,2.699339e+08,5.535587e+00,2.637365e+08,...,1.709646e+08,5.984108e+07,1.648001e+01,8.056551e+07,2.449808e+01,4.856147e+07,9.492283e+07,3.091887e+07,7.457657e+07,3.857762e+07
4,6.782679e+07,2.654722e+08,5.826158e+07,2.990796e+08,6.113911e+07,1.549960e+08,7.623457e+07,2.873457e+08,5.729947e+07,2.525232e+08,...,2.042090e+08,7.273202e+07,1.419164e+08,8.608843e+07,1.271596e+08,6.372965e+00,9.668526e+07,4.663450e+07,7.871942e+07,4.549146e+07


### Verify if imputation ocurred currectly

In [14]:
orig_imputed = imputed_df[~nan_mask]
orig_original = original_df[~nan_mask]
preservou_originais = orig_imputed.equals(orig_original)

if preservou_originais:
    print("✅ Todos os valores originais foram preservados nos dados imputados.")
else:
    print("❌ Os dados originais foram modificados em pelo menos um ponto!")
    alterados = (imputed_df != original_df) & (~nan_mask)
    print(alterados.sum().sum(), "pontos originais foram alterados.")

# 2. Verificar se restou algum NaN no resultado da imputação
tem_nan_apos_imputacao = imputed_df.isna().any().any()

if tem_nan_apos_imputacao:
    print("⚠️ Ainda existem valores NaN no DataFrame imputado.")
else:
    print("✅ Não existem valores NaN no DataFrame imputado.")


✅ Todos os valores originais foram preservados nos dados imputados.
✅ Não existem valores NaN no DataFrame imputado.


### Evaluate

In [13]:

rmse_imputed = np.sqrt(
    np.mean((original_df.values/1000000 - imputed_df.values/1000000) ** 2)
)

amplitude = (
    df_throughput['throughput'].values.max() - df_throughput['throughput'].values.min()
)/1000000
nrmse_imputed = rmse_imputed / amplitude if amplitude != 0 else np.nan

# Step 5: Print results
print(f"RMSE (Imputed Data): {rmse_imputed:.4f}")
print(f"NRMSE (Imputed Data): {(nrmse_imputed * 100):.4f}%")


RMSE (Imputed Data): 56.2762
NRMSE (Imputed Data): 14.8354%


### Soft Impute with grid search 

In [ ]:
df = pd.read_csv("institution_subnets/institution_subnets/agg_1_hour/0.csv")

df["throughput"] = df["n_bytes"] / 3600 * 8  # Para throughput em bps

df_throughput = df[["throughput"]]
